In [1]:
%reload_ext xmemo_kernel_extension

In [2]:
import json
import pandas as pd
import pickle
import openai
import functools
import numpy as np
import concurrent.futures

In [3]:
!git xet materialize data/squad2.json

Materializing 1 file(s)...
Done


In [4]:
d = pd.read_json('data/squad2.json')['data']
all_paragraphs = [paragraph['context'] for ent in d for paragraph in ent['paragraphs']]

In [5]:
%%xmemo input=all_paragraphs output=all_qa

# Ok. list comprehension is still possible but it gets a little obnoxious.
all_qa = []
paragraph_id = 0
for ent in d:
    for paragraph in ent['paragraphs']:
        for qa in paragraph['qas']:
            if len(qa['answers']) > 0 and qa['is_impossible'] == False:
                all_qa.append((paragraph_id, qa['question'], qa['answers'][0]['text']))
        paragraph_id += 1

/Users/ylow/xetrepos/RagIRBench2/xmemo/e06196685b0c2069ecd0b9336662b29b395d8b8fc7c1b9eb2117f1c1f2804e7a.pickle
b"\x80\x04\x95-\x00\x01\x00\x00\x00\x00\x00]\x94(K\x00\x8c(When did Beyonce start becoming popular?\x94\x8c\x11in the late 1990s\x94\x87\x94K\x00\x8c:What areas did Beyonce compete in when she was growing up?\x94\x8c\x13singing and dancing\x94\x87\x94K\x00\x8c@When did Beyonce leave Destiny's Child and become a solo singer?\x94\x8c\x042003\x94\x87\x94K\x00\x8c-In what city and state did Beyonce  grow up? \x94\x8c\x0eHouston, Texas\x94\x87\x94K\x00\x8c*In which decade did Beyonce become famous?\x94\x8c\n"


In [ ]:
all_qa

In [ ]:
!git xet materialize data/openai_paragraphs_embeddings.npy 
!git xet materialize data/openai_qn_embeddings.npy

In [ ]:
from sklearn.neighbors import NearestNeighbors
par_embeddings_arr = np.load('data/openai_paragraphs_embeddings.npy')
qn_embeddings_arr = np.load('data/openai_qn_embeddings.npy')

In [ ]:
%%xmemo input=par_embeddings_arr,qn_embeddings_arr output=knn
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree', n_jobs=-1)
nbrs.fit(par_embeddings_arr)
knn = nbrs.kneighbors(qn_embeddings_arr)

In [ ]:
knn